# Token classification (PyTorch)

Install the Transformers and Datasets libraries to run this notebook.

In [5]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

     |████████████████████████████████| 306 kB 1.5 MB/s 
     |████████████████████████████████| 3.4 MB 28.8 MB/s 
     |████████████████████████████████| 61 kB 611 kB/s 
     |████████████████████████████████| 243 kB 70.6 MB/s 
     |████████████████████████████████| 1.1 MB 58.1 MB/s 
     |████████████████████████████████| 132 kB 84.9 MB/s 
     |████████████████████████████████| 192 kB 77.8 MB/s 
     |████████████████████████████████| 160 kB 75.5 MB/s 
     |████████████████████████████████| 271 kB 78.8 MB/s 
     |████████████████████████████████| 596 kB 64.4 MB/s 
     |████████████████████████████████| 895 kB 59.0 MB/s 
     |████████████████████████████████| 3.3 MB 58.5 MB/s 
     |████████████████████████████████| 1.2 MB 62.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-

You will need to setup git, adapt your email and name in the following cell.

In [6]:
!git config --global user.email "naturalchyy@gmail.com"
!git config --global user.name "chiamaka"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [7]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [8]:
from datasets import load_dataset

raw_datasets = load_dataset('masakhaner', 'ibo')

Downloading:   0%|          | 0.00/2.75k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.25k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/88.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset masakhaner downloaded and prepared to /root/.cache/huggingface/datasets/masakhaner/ibo/1.0.0/e61b24903076a3af7682855beebb820ec64edad0d6787b148c473694592d10b3. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2235
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 320
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 638
    })
})

In [10]:
raw_datasets["train"][0]["tokens"]

['Ike',
 'ịda',
 'jụụ',
 'otụ',
 'nkeji',
 'banyere',
 'oke',
 'ogbugbu',
 'na',
 '-',
 'eme',
 "n'ala",
 'Naijiria',
 'agwụla',
 'Ekweremmadụ']

In [11]:
raw_datasets["train"][0]["ner_tags"]

[0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1]

In [12]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE'], names_file=None, id=None), length=-1, id=None)

In [13]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE']

In [14]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

Ike ịda jụụ otụ    nkeji  banyere oke ogbugbu na - eme n'ala Naijiria agwụla Ekweremmadụ 
O   O   O   B-DATE I-DATE O       O   O       O  O O   O     B-LOC    O      B-PER       


In [15]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [16]:
MODEL_PATH = "/content/gdrive/MyDrive/IBO_BETA/IgboBert"
checkpoint = "MODEL_PATH"

In [17]:
from transformers import AutoTokenizer

model_checkpoint = MODEL_PATH
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,add_prefix_space=True)

In [18]:
tokenizer.is_fast

True

In [19]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['<s>',
 'ĠIke',
 'Ġá»ĭda',
 'Ġjá»¥á»¥',
 'Ġotá»¥',
 'Ġnkeji',
 'Ġbanyere',
 'Ġoke',
 'Ġogbugbu',
 'Ġna',
 'Ġ-',
 'Ġeme',
 'Ġn',
 "'",
 'ala',
 'ĠNaijiria',
 'Ġagwá»¥la',
 'ĠEkweremmadá»¥',
 '</s>']

In [20]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, None]

In [21]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [22]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1]
[-100, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1, -100]


In [23]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [24]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

  0%|          | 0/3 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [25]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [26]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    0,    0,    0,    7,    8,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    5,    0,    1, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    5,    0,    1,
            2,    0,    0,    0,    0,    0,    0,    0,    0,    7,    8,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    5,
            0,    0,    0,    0, -100]])

In [27]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1, -100]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, -100]


In [28]:
!pip install seqeval

     |████████████████████████████████| 43 kB 1.4 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=eb50c083e5eea8be47aeb56ef47359a5049e10279ff8bf059f7719870065008e
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [29]:
from datasets import load_metric

metric = load_metric("seqeval")

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

In [30]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['O',
 'O',
 'O',
 'B-DATE',
 'I-DATE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'B-PER']

In [31]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'DATE': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'LOC': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'PER': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

In [32]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [33]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [34]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at /content/gdrive/MyDrive/IBO_BETA/IgboBert were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at /content/gdrive/MyDrive/IBO_BETA/IgboBert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model 

In [35]:
model.config.num_labels

9

In [36]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [37]:
from transformers import TrainingArguments

args = TrainingArguments(
    "Iboberti-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    #push_to_hub=True,
)

In [38]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

***** Running training *****
  Num examples = 2235
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1400


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.187419,0.582865,0.691667,0.632622,0.936535
2,0.231000,0.157946,0.675425,0.728333,0.700882,0.947206
3,0.231000,0.152899,0.685583,0.745000,0.714058,0.949874
4,0.094000,0.150583,0.690076,0.753333,0.720319,0.952120
5,0.094000,0.152252,0.682635,0.760000,0.719243,0.951278


***** Running Evaluation *****
  Num examples = 320
  Batch size = 8
Saving model checkpoint to Iboberti-finetuned-ner/checkpoint-280
Configuration saved in Iboberti-finetuned-ner/checkpoint-280/config.json
Model weights saved in Iboberti-finetuned-ner/checkpoint-280/pytorch_model.bin
tokenizer config file saved in Iboberti-finetuned-ner/checkpoint-280/tokenizer_config.json
Special tokens file saved in Iboberti-finetuned-ner/checkpoint-280/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 320
  Batch size = 8
Saving model checkpoint to Iboberti-finetuned-ner/checkpoint-560
Configuration saved in Iboberti-finetuned-ner/checkpoint-560/config.json
Model weights saved in Iboberti-finetuned-ner/checkpoint-560/pytorch_model.bin
tokenizer config file saved in Iboberti-finetuned-ner/checkpoint-560/tokenizer_config.json
Special tokens file saved in Iboberti-finetuned-ner/checkpoint-560/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 320
  Batch size

TrainOutput(global_step=1400, training_loss=0.1323652458190918, metrics={'train_runtime': 81.2346, 'train_samples_per_second': 137.565, 'train_steps_per_second': 17.234, 'total_flos': 125135573326344.0, 'train_loss': 0.1323652458190918, 'epoch': 5.0})

In [ ]:
#trainer.push_to_hub(commit_message="Training complete")

'https://huggingface.co/sgugger/bert-finetuned-ner/commit/26ab21e5b1568f9afeccdaed2d8715f571d786ed'

In [39]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [40]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

loading configuration file /content/gdrive/MyDrive/IBO_BETA/IgboBert/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/gdrive/MyDrive/IBO_BETA/IgboBert",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-DATE",
    "8": "I-DATE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-DATE": "7",
    "B-LOC": "5",
    "B-ORG": "3",
    "B-PER": "1",
    "I-DATE": "8",
    "I-LOC": "6",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "pos

In [41]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [42]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [43]:
from transformers import get_scheduler

num_train_epochs = 5
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-ner-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

'sgugger/bert-finetuned-ner-accelerate'

In [ ]:
output_dir = "bert-finetuned-ner-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

In [44]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [45]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/1400 [00:00<?, ?it/s]

epoch 0: {'precision': 0.7016666666666667, 'recall': 0.6022889842632332, 'f1': 0.6481909160892994, 'accuracy': 0.9328840213423196}


NameError: ignored

In [ ]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

In [47]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "/content/Iboberti-finetuned-ner/checkpoint-1120"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("Aha onyeisi Ụlọọrụ NNPC bu Obianọ.")

loading configuration file /content/Iboberti-finetuned-ner/checkpoint-1120/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/Iboberti-finetuned-ner/checkpoint-1120",
  "architectures": [
    "RobertaForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-DATE",
    "8": "I-DATE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-DATE": "7",
    "B-LOC": "5",
    "B-ORG": "3",
    "B-PER": "1",
    "I-DATE": "8",
    "I-LOC": "6",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "p

[{'end': 33,
  'entity_group': 'PER',
  'score': 0.8210868,
  'start': 27,
  'word': ' Obianọ'}]

In [48]:
nlp = pipeline(
    "ner",
    model=model_checkpoint
)

loading configuration file /content/Iboberti-finetuned-ner/checkpoint-1120/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/Iboberti-finetuned-ner/checkpoint-1120",
  "architectures": [
    "RobertaForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-DATE",
    "8": "I-DATE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-DATE": "7",
    "B-LOC": "5",
    "B-ORG": "3",
    "B-PER": "1",
    "I-DATE": "8",
    "I-LOC": "6",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "p

In [49]:
nlp("Onyeisi Ụlọọrụ NNPC.")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[]